#### Download and train yolo11n on coco8

In [ ]:
import numpy as np
from ultralytics import YOLO
import supervision as sv
import cv2
import time

    # Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

    # Train the model on the COCO8 dataset for 100 epochs
train_results = model.train(
    data="coco8.yaml",  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
)

    # Evaluate the model's performance on the validation set
metrics = model.val()

    # Perform object detection on an image
results = model("https://ultralytics.com/images/bus.jpg")  # Predict on an image
results[0].show()  # Display results

    # Export the model to ONNX format for deployment
#path = model.export(format="onnx")  # Returns the path to the exported model


#### Train on 57 annotated image sets

In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 dataset for 100 epochs
train_results = model.train( data="./datasets/10_03_25_57imgs/data.yaml", # Path to dataset configuration file 
                             epochs=100, # Number of training epochs 
                             imgsz=640, # Image size for training 
                           )


#### Train on 137 augmented/annotated image sets

In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 dataset for 100 epochs
train_results = model.train( data="./datasets/10_03_25__137imgs_augmented//data.yaml", # Path to dataset configuration file 
                             epochs=100, # Number of training epochs 
                             imgsz=640, # Image size for training 
                           )

### Inference on videostream

In [ ]:
import numpy as np
from ultralytics import YOLO
import supervision as sv
import cv2
import time
	
# Load a pretrained YOLO11n model
model = YOLO("./weights/best.pt")

inference = model("./datasets/10_03_25_57imgs/test/images/IMG_0432_JPG.rf.051c3c92a77f62b6609178e5f7d85fbb.jpg")
result = inference[0]
detections = sv.Detections.from_ultralytics (result)
detections

### Path Detection - Inference/Bounding

In [ ]:

vid_cap = cv2.VideoCapture('./datasets/recording/IMG_5795.MOV')
while (vid_cap.isOpened()):
    _, frame = vid_cap.read()
    inference = model( frame, )[0]

    im_bgr = inference.plot()  # BGR-order numpy array
    #im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    cv2.imshow('result', cv2.resize(im_bgr, (640, 640)) )

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
vid_cap.release()
cv2.destroyAllWindows()

### Centroids On Image

In [ ]:
import cv2
from PIL import Image

radius = 5                    # Radius in pixels
color = (255, 0, 255)             # BGR color (Green in this case)
thickness = 2                   # Thickness of the circle line (-1 for a filled circle)
thickness = -1 # -1 indicates a filled circle   


vid_cap = cv2.VideoCapture('./datasets/recording/IMG_5795.MOV')
while (vid_cap.isOpened()):
    _, frame = vid_cap.read()
    frame = cv2.resize(frame, (640, 640))
    inference = model(frame, verbose=False)[0]
   
    detections = sv.Detections.from_ultralytics( inference)
    walkway = detections[detections.class_id == 4] #walkway
    if len(walkway) < 3:
        continue
    
    centroid_list = [((b1[0] + b1[2])/2, (b1[1] + b1[3])/2) for b1 in walkway.xyxy ]
    centroid_x = [ int(x[0]) for x in centroid_list]
    centroid_y = [ int(y[1]) for y in centroid_list]
    
    for i, x in enumerate(centroid_x):
        #Define circle parameters
        center_coordinates = (x, centroid_y[i])  # (x, y) - center of the circle
        # 3. Draw the circle on the image
        cv2.circle(frame, center_coordinates, radius, color, thickness)
    
    # 4. Display the image with the centroids
    cv2.imshow('result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
vid_cap.release()

cv2.destroyAllWindows()

### Mean of Centroids/Frame On Image

In [ ]:
data_centroid_x = np.empty([0])
data_centroid_y = np.empty([0])

data_centroid_list_x = np.empty([0])
data_centroid_list_y = np.empty([0])
data_walkways = np.empty([0])

model = YOLO("./weights/best.pt")
vid_cap = cv2.VideoCapture('./datasets/recording/IMG_5795.MOV')
total_frames = 0

start_time = time.perf_counter()

while (vid_cap.isOpened()):
   
    # Read a frame from the video
    ret, frame = vid_cap.read()

    # If ret is False, it means the video has ended
    if not ret:
        print("Video has finished.")
        break
        
    frame = cv2.resize(frame, (640, 640))
    inference = model(frame, verbose=False)[0]
   
    walkway = sv.Detections.from_ultralytics( inference)
    #walkway = detections[(detections.class_id == 4) & (detections.confidence > 0.5)] #walkway
    
    total_frames += 1
    
    if len(walkway) < 3:
        continue      
    
    data_walkways = np.append(data_walkways, len(walkway))
    
    centroid_list = [((b1[0] + b1[2])/2, (b1[1] + b1[3])/2) for b1 in walkway.xyxy ]
    centroid_list.sort(key=lambda p: p[0])
    
    centroid_x = [ x[0] for x in centroid_list]
    centroid_y = [ y[1] for y in centroid_list]

    data_centroid_x = np.append(data_centroid_x, (centroid_x))
    data_centroid_y = np.append(data_centroid_y, (centroid_y))
    
    data_centroid_list_x = np.append(data_centroid_list_x, np.mean(centroid_x))
    data_centroid_list_y = np.append(data_centroid_list_y, np.mean(centroid_y))
          

    # Visualize centroids
    cv2.circle(frame, (int(data_centroid_list_x[-1]), int(data_centroid_list_y[-1])), 5, (0, 0, 255), -1)

    # 4. Display the image with the centroids
    cv2.imshow('result', frame)
       
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

end_time = time.perf_counter()
print("Time taken {}s for {} frames".format(round(end_time - start_time, 2), total_frames))
print("frames {} per seconds".format(total_frames/round(end_time - start_time, 2)))

vid_cap.release()
cv2.destroyAllWindows()
